In [2]:
from google.colab import drive
import os
drive.mount('/content/drive')

os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive"

!kaggle datasets download -d shubhampatel231/name-classification

!unzip \*.zip && rm *.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset URL: https://www.kaggle.com/datasets/shubhampatel231/name-classification
License(s): MIT
100% 61.7k/61.7k [00:00<00:00, 405kB/s]
100% 61.7k/61.7k [00:00<00:00, 404kB/s]
Archive:  name-classification.zip
  inflating: Arabic.txt              
  inflating: Chinese.txt             
  inflating: Czech.txt               
  inflating: Dutch.txt               
  inflating: English.txt             
  inflating: French.txt              
  inflating: German.txt              
  inflating: Greek.txt               
  inflating: Irish.txt               
  inflating: Italian.txt             
  inflating: Japanese.txt            
  inflating: Korean.txt              
  inflating: Polish.txt              
  inflating: Portuguese.txt          
  inflating: Russian.txt             
  inflating: Scottish.txt            
  inflating: Spanish.txt             
  inflating: V

In [4]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from utils import ALL_LETTERS, N_LETTERS
from utils import load_data, letter_to_tensor, line_to_tensor, random_training_example

In [5]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(RNN, self).__init__()

    self.hidden_size = hidden_size
    self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
    self.i2o = nn.Linear(input_size + hidden_size, output_size)
    self.softmax = nn.LogSoftmax(dim = 1)

  def forward(self, input_tensor, hidden_tensor):
    combined = torch.cat(input_tensor, hidden_tensor, 1)

    hidden = self.i2h(combined)
    output = self.i2o(combined)
    output = self.softmax(output)
    return output, hidden

  def init_hidden(self):
    return torch.zeros(1, self.hidden_size)

In [6]:
category_lines, all_categories = load_data()
n_categories = len(all_categories)

In [7]:
n_hidden = 128
rnn = RNN(N_LETTERS, n_hidden, n_categories)

In [8]:
input_tensor = letter_to_tensor('A')
hidden_tensor = rnn.init_hidden()

output, next_hidden = rnn(input_tensor, hidden_tensor)

TypeError: cat() received an invalid combination of arguments - got (Tensor, Tensor, int), but expected one of:
 * (tuple of Tensors tensors, int dim, *, Tensor out)
 * (tuple of Tensors tensors, name dim, *, Tensor out)


In [9]:
def category_from_output(output):
  category_idx = torch.argmax(output).item()
  return all_categories[category_idx]

In [10]:
criterion = nn.NLLLoss()
optimizer = torch.optim.SGD(rnn.parameters(), lr = 0.005)

In [11]:
def train(line_tensor, category_tensor):
  hidden = rnn.init_hidden()

  for i in range(line_tensor.size()[0]):
    output, hidden = rnn(line_tensor[i], hidden)

  loss = criterion(output, category_tensor)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  return output, loss.item()

In [12]:
current_loss = 0
all_losses = []
plot_steps, print_steps = 1000, 5000
n_iters = 100000

for i in range(n_iters):
  category, line, category_tensor, line_tensor = random_training_example(category_lines, all_categories)
  output, loss = train(line_tensor, category_tensor)
  current_loss += loss

  if (i+1) % plot_steps == 0:
    all_losses.append(current_loss / plot_steps)
    current_loss = 0
  if (i+1) % print_steps == 0:
    guess = category_from_output(output)
    correct = "CORRECT" if guess == categors else f"WRONG ({category})"
    print(f'{i} {i/n_iters*100} {loss:.4f} {line}/{guess} {correct}')

TypeError: cat() received an invalid combination of arguments - got (Tensor, Tensor, int), but expected one of:
 * (tuple of Tensors tensors, int dim, *, Tensor out)
 * (tuple of Tensors tensors, name dim, *, Tensor out)


In [ ]:
plt.figure()
plt.plot(all_losses)
plt.show()

In [13]:
def predict(input_line):
  print(f'\n {input_line}')
  with torch.no_grad():
    line_tensor = line_to_tensor(input_line)
    hidden = rnn.init_hidden()

    for i in range(line_tensor.size()[0]):
      output, hidden = rnn(line_tensor[i], hidden)
    guess = category_from_output(output)
    print(guess)

In [14]:
while True:
  sentence = input("Giriş: ")
  if sentence == "quit":
    break
  predict(sentence)

Giriş: English

 English


TypeError: cat() received an invalid combination of arguments - got (Tensor, Tensor, int), but expected one of:
 * (tuple of Tensors tensors, int dim, *, Tensor out)
 * (tuple of Tensors tensors, name dim, *, Tensor out)
